Data Collection

In [279]:
cd C:\\Users\\bunny #### Setting current directory ###

C:\Users\bunny


In [280]:
### Importing numpy and requests packages ###

import numpy as np
import requests

### Requests sent to index page of the given link ###
page = requests.get("http://mlg.ucd.ie/modules/COMP41680/archive/index.html")
page

<Response [200]>

In [281]:
### Creating a directory to store all the data files of articles ###
import os

if not os.path.exists("C:/Users/bunny/data_files"):
    os.makedirs("C:/Users/bunny/data_files")

In the below cell, we are extracting all the content from the articles using Beautiful soup and storing each article in a different text file.  

In [289]:
### Exception handling for connection error ### 
from requests.exceptions import ConnectionError

### Importing Beautiful soup package and using it to get the content of index page parsing through html ###

from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

### Finding the a tags in the class main in the content that was extracted using beautiful soup from index page ###
article_name_list = soup.find(class_='main')
article_name_list_items = article_name_list.find_all('a')

### Creating numpy arrays to store links and one final list to store text ###
links_array = np.array([])
months_array = np.array([])
final_list=[]
title_list=[]
### First for loop is to iterate through the 12 months links i.e; from jan to dec ###
for article_name in article_name_list_items:

    links = "http://mlg.ucd.ie/modules/COMP41680/archive/" + article_name.get('href') ### getting links through href tag ###
    links_array = np.append(links_array, links)  ### storing all the first 12 months in an array ###
    #print(links)
    
    ### Sending request to link belonging to each and every month ###
    page_months = requests.get(links)
    soup_months = BeautifulSoup(page_months.content, 'html.parser') ### Getting contents of the above links ###
    
    ### Finding the a tags in class main to get href tags and extract links of articles ###
    months_list = soup_months.find(class_ ='main')
    months_list_items = months_list.find_all('a')
    
    ### This for loop is to iterate through all the articles in each month
    for articles in months_list_items:
        month_links = "http://mlg.ucd.ie/modules/COMP41680/archive/" + articles.get('href')
        months_array = np.append(months_array, month_links) ### storing all the articles in an array ###
        #print(month_links)
        
        ### sending requests to article links ### 
        page_data = requests.get(month_links)
        soup_article = BeautifulSoup(page_data.content, 'html.parser') ### extracting article page content ###
        
        ### Finding all the 'p' tags in the articles to get content ###
        text_iter = soup_article.findAll('p')
        
        ### Exception Handling to avoid connection error ###
        try:
            finaltext = ""
            
            ### This for loop is to go through every 'p' tag text ###
            for tag in text_iter:
                finaltext = finaltext + tag.getText()
            final_list.append(finaltext)
                ### Adding every word to final text ###
                ### Storing every article in a different text file ###
            try:
                filename = finaltext.split(" ")[0] + '_'+ strftime("%Y-%m-%d %H:%M:%S", gmtime()).split(" ")[1].replace(":", "_")
                f = open(os.getcwd()+"\\"+"data_files\\"+filename+".txt", 'w+')
                f.write(finaltext)
                f.close()
            ### Exception handling to pass these errors
            except UnicodeEncodeError:
                pass
        except ConnectionError:
            pass

This code is to get category and title of each article using beautiful soup through html parser and storing them in the CSV files.

In [262]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

### Finding the a tags in the class main in the content that was extracted using beautiful soup from index page ###
article_name_list = soup.find(class_='main')
article_name_list_items = article_name_list.find_all('a')

### Creating numpy arrays to store links and one final list to store text ###
final_list_title=[]
category_list=[]
### First for loop is to iterate through the 12 months links i.e; from jan to dec ###
for article_name in article_name_list_items:

    links = "http://mlg.ucd.ie/modules/COMP41680/archive/" + article_name.get('href')
    
        ### Sending request to link belonging to each and every month ###
    page_months = requests.get(links)
    soup_months = BeautifulSoup(page_months.content, 'html.parser') ### Getting contents of the above links ###
    
    #category is a list of html tags with the category
    category=soup_months.find_all('td',class_="category")
    for idx, cat in enumerate(category):
        cat = str(cat)
        #extract the category by extracting <, > characters
        temp_string = cat.split(">")[1]
        index = temp_string.find("<")
        final_string = temp_string[0:index]
        final_string = str(final_string.replace('\xa0',""))

        if(len(final_string)<13 ):
            #replace the category list with proper categories. 
            category[idx] = final_string    
    category_list.append(category)    
    
    #extract the titles
    title_list = soup_months.findAll('a')
    final_title_list = []
    for idx, title in enumerate(title_list):
        if(idx not in list(range(len(title_list)-4, len(title_list)))):
            title = str(title)
            start_index = title.find(">")
            end_index = title.find("<", int(start_index) + 1)
            extract_string = title[start_index+1:end_index]
            #print(extract_string)
            final_title_list.append(extract_string)
    final_list_title.append(final_title_list)
        
    #final_title_list
#category

This code is to convert final title and category list of lists to a flat list and convert them to a pandas series dataframe and save them in the csv file.  

In [270]:
import pandas as pd
flat_title_list = [item for sublist in final_list_title for item in sublist]
df = pd.DataFrame(flat_title_list)
df.to_csv('title.csv', index=False)

flat_category_list = [item for sublist in category_list for item in sublist]
df = pd.DataFrame(flat_category_list)
df.to_csv('category.csv', index=False)

This is to put all the extracted article data in one text file 

In [291]:
### Importing pandas to convert the final list to a data frame and storing the final list in a .txt file in the given directory ###

import pandas as pd
df = pd.DataFrame(final_list)
df.to_csv('article_text.csv', index=False)

In [290]:
len(final_list)

1408